This Notebook will store cleaned data in a SQLite database, and explore plotting.

Importing libraries and clean tree and air dataframes:

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import sqlite3
import tabulate

In [26]:
air = pd.read_csv("Air_quality_by_CD.csv")
air.head()

,id,Geo_Join_ID,Time_Period,Name,Data_Value,Measure_Info,Indicator_ID
0,1,304.0,Summer 2015,Nitrogen dioxide (NO2),15.5,ppb,375
1,2,410.0,Winter 2015-16,Nitrogen dioxide (NO2),22.5,ppb,375
2,3,301.0,Summer 2015,Nitrogen dioxide (NO2),17.0,ppb,375
3,4,409.0,Summer 2015,Nitrogen dioxide (NO2),14.4,ppb,375
4,5,304.0,Annual Average 2015,Nitrogen dioxide (NO2),20.3,ppb,375


In [27]:
tree = pd.read_csv("tree_summary_by_cb.csv")
tree.head()

,tree_id,cb_num,tree_count,avg_health_score,dbh_mean,dbh_max
0,1,101,2297,2.7101,7.096,88
1,2,102,4833,2.6739,8.898,33
2,3,103,4709,2.7084,8.449,32
3,4,104,4419,2.7798,7.941,38
4,5,105,2016,2.6037,6.375,21


In [28]:
air_and_tree = pd.read_csv("combined_air_and_tree.csv")
air_and_tree.head()

,id,Geo_Join_ID,Time_Period,Name,Data_Value,Measure_Info,Indicator_ID,tree_id,cb_num,tree_count,avg_health_score,dbh_mean,dbh_max
0,1,304.0,Summer 2015,Nitrogen dioxide (NO2),15.5,ppb,375,28,304,6817,2.7081,8.918,52
1,2,410.0,Winter 2015-16,Nitrogen dioxide (NO2),22.5,ppb,375,52,410,14353,2.7888,14.260,64
2,3,301.0,Summer 2015,Nitrogen dioxide (NO2),17.0,ppb,375,25,301,11542,2.8084,9.234,151
3,4,409.0,Summer 2015,Nitrogen dioxide (NO2),14.4,ppb,375,51,409,10878,2.7726,14.965,132
4,5,304.0,Annual Average 2015,Nitrogen dioxide (NO2),20.3,ppb,375,28,304,6817,2.7081,8.918,52


Creating a SQLite database:

In [29]:
air_df = pd.read_csv("Air_quality_by_CD.csv")
tree_df = pd.read_csv("tree_summary_by_cb.csv")

In [35]:
#Create SQLite database and tables
conn = sqlite3.connect("air_and_tree.db")
cursor = conn.cursor()

#create table air
cursor.execute("""
CREATE TABLE IF NOT EXISTS air (
    id INTREGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Geo_Join_ID TEXT NOT NULL,
    Time_Period TEXT NOT NULL,
    Data_Value REAL NOT NULL
);        
""")

#create table tree
cursor.execute(""" 
CREATE TABLE IF NOT EXISTS tree (
    tree_id INTEGER PRIMARY KEY,
    cb_num TEXT NOT NULL,
    dbh_mean REAL NOT NULL,
    dbh_max INTEGER NOT NULL,
    tree_count INTEGER NOT NULL,
    avg_health_score REAL NOT NULL
);             
""")

#insert data into tables
air_df.to_sql("air", conn, if_exists="replace", index=False)
tree_df.to_sql("tree", conn, if_exists="replace", index=False)

#verify data was inserted
print("Air Data Table: \n")
print(pd.read_sql_query("SELECT * FROM air LIMIT 5", conn))

print("Tree Data Table: \n")
print(pd.read_sql_query("SELECT * FROM tree", conn))



Air Data Table: 

   id  Geo_Join_ID          Time_Period                    Name  Data_Value  \
0   1        304.0          Summer 2015  Nitrogen dioxide (NO2)        15.5   
1   2        410.0       Winter 2015-16  Nitrogen dioxide (NO2)        22.5   
2   3        301.0          Summer 2015  Nitrogen dioxide (NO2)        17.0   
3   4        409.0          Summer 2015  Nitrogen dioxide (NO2)        14.4   
4   5        304.0  Annual Average 2015  Nitrogen dioxide (NO2)        20.3   

  Measure_Info  Indicator_ID  
0          ppb           375  
1          ppb           375  
2          ppb           375  
3          ppb           375  
4          ppb           375  
Tree Data Table: 

    tree_id  cb_num  tree_count  avg_health_score  dbh_mean  dbh_max
0         1     101        2297            2.7101     7.096       88
1         2     102        4833            2.6739     8.898       33
2         3     103        4709            2.7084     8.449       32
3         4     104       

In [36]:
#setting up a query function
def query(query: str):
    return pd.read_sql(query, conn)

In [41]:
all = """SELECT * FROM tree
        LIMIT 10"""
query(all)

,tree_id,cb_num,tree_count,avg_health_score,dbh_mean,dbh_max
0,1,101,2297,2.7101,7.096,88
1,2,102,4833,2.6739,8.898,33
2,3,103,4709,2.7084,8.449,32
3,4,104,4419,2.7798,7.941,38
4,5,105,2016,2.6037,6.375,21
5,6,106,4864,2.7052,8.742,199
6,7,107,8332,2.7751,9.606,54
7,8,108,8428,2.6362,9.334,74
8,9,109,4755,2.7159,9.491,109
9,10,110,5632,2.7376,8.564,228


In [49]:
nitrogen_dioxide = """SELECT Geo_Join_ID, Name, Data_Value 
                        FROM air
                        WHERE Name = 'Nitrogen dioxide (NO2)'
                        AND Time_Period = 'Summer 2015';
                        """
query(nitrogen_dioxide)

,Geo_Join_ID,Name,Data_Value
0,304.0,Nitrogen dioxide (NO2),15.5
1,301.0,Nitrogen dioxide (NO2),17.0
2,409.0,Nitrogen dioxide (NO2),14.4
3,201.0,Nitrogen dioxide (NO2),17.4
4,104.0,Nitrogen dioxide (NO2),23.4
5,413.0,Nitrogen dioxide (NO2),12.1
6,105.0,Nitrogen dioxide (NO2),29.7
7,209.0,Nitrogen dioxide (NO2),15.8
8,411.0,Nitrogen dioxide (NO2),13.0
9,211.0,Nitrogen dioxide (NO2),15.7
